# Competition #1: Milestone Report
## Research Question
   *What are the determinants of IPO underpricing phenomena?*
## Business Understanding
   When presented with this business question, our first task was to better understand the question at hand. We started by researching the IPO underpricing phenomena to familiarize ourselves with the business question and standard IPO terminology. Underpricing occurs when the price of an initial public offering (IPO) is lower than the price of the first trade.
## Data Understanding   
   Our next step was to study the data dictionary provided. What variables were we provided with to predict the determinants of underpricing? The dataset provided had 682 records, and 22 attributes. Variable data types included Continuous/Float, Binary/Boolean, and Text. We aimed to get a better understanding of the types of variables provided along with their definitions, not only to help us determine whether instances in our dataset seemed incorrect, but also so that we may come up with hypotheses of which variables we thought may make the best predictors. Once we gained a better idea of the data that was provided via the data dictionary and general research, we moved on to inspecting the actual dataset. We started this task by reading the data into python, previewing the loaded dataframe, inspecting the datatypes each variable mapped to, running descriptive statistics on the dataset, and finally creating histograms for each variable.

In [1]:
# Import Packages
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
%matplotlib inline

# Read in Data file and define NaN values
ipo_data = pd.read_excel("Competation #1 Raw Data_New.xlsx",header=0,na_values='-' )

# Run Descriptive Statistics on Data File
ipo_data.describe()

C:\Users\danielle.ezzo\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,C4,C5,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
count,677.000000,672.000000,672.000000,660.000000,660.000000,660.000000,646.000000,660.000000,6.760000e+02,6.760000e+02,610.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000
mean,13.837666,15.481190,13.515045,25.934766,149.728788,0.859091,1.788904,0.007282,4.935776e+07,1.241519e+07,500.459962,465.634361,12758.606461,11395.844347,294.353891,679.220264,68.421439,120.104258,144.759178
std,6.053731,6.653429,5.835646,73.234948,152.817467,0.348192,162.666532,0.033318,1.043764e+08,2.512855e+07,1648.337634,175.741647,5449.644597,4839.670179,121.532637,472.914323,39.096525,84.828959,69.276285
min,3.000000,0.000000,3.000000,0.000000,10.000000,0.000000,-786.239000,-0.162352,3.693227e+06,5.250000e+05,0.074000,132.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,20.000000,26.000000
25%,10.000000,12.500000,11.000000,11.000000,85.000000,1.000000,-0.852500,-0.013927,1.871417e+07,5.000000e+06,37.245750,351.000000,9195.000000,8162.000000,213.000000,462.000000,45.000000,73.000000,100.000000
50%,13.500000,15.000000,13.000000,14.845000,107.000000,1.000000,0.010000,0.009125,2.740018e+07,7.398704e+06,103.833000,444.000000,12045.000000,10785.000000,279.000000,624.000000,60.000000,100.000000,134.000000
75%,17.000000,17.000000,15.000000,20.485000,155.250000,1.000000,0.470000,0.031571,4.980786e+07,1.200000e+07,331.138000,551.000000,15241.000000,13760.000000,354.000000,795.000000,85.000000,142.000000,173.000000
max,85.000000,135.000000,108.000000,1159.200562,2087.000000,1.000000,3864.500000,0.092896,2.138085e+09,4.212336e+08,30683.000000,1750.000000,49056.000000,43952.000000,1058.000000,10277.000000,309.000000,944.000000,883.000000


The resulting chart above led us to several immediate takeaways:
   * C2, or the Top Tier Dummy variable, should be converted to a boolean
   * P(H), was displaying a minimum of 0. Given that this is the upper bound of the IPO price range, this value must be incorrect
   * Many attributes had maximum values that seemed dramatically high given the mean and interquartile range provided
   * There may be instances where P(L) is higher than P(H), which would be incorrect, as the upper bound of the price range could not be lower than the lower bound of the range
   * Our target and control variables would need to be created

These immediate takeaways gave us a better foundation of where our data stood, and gave us ideas of where to next investigate anomalies, outliers within each attribute and record.

 [Variable Histograms](https://github.com/ConorFeeney/IS540-Code/blob/master/Python%20for%20Comp%201/Attribute%20Normality.png) 

## Data Preperation 
Once we gained a better understanding of our data, we were able to move forward with preparing the data based on our findings. We decided to utlize multiple methods in each of the suggested tasks provided, as well as some others.

### Imputation
When replacing missing values, our methods depended on the data type of the specific variable in question. For descriptive attributes like I3, we conducted research to find the correct identification numbers and then manually updated our dataset. For Continuous/Float variables, we used a python script to replace the missing variables with their respective means, as this is an often using procedure when dealing with missing values to replace with either the mean or median. For the purposes of this report, we arbitrarily choose the mean for our benchmark model.

In [1]:
#Replace NaN values with mean
ipo_data=ipo_data.fillna(ipo_data.mean())

NameError: name 'ipo_data' is not defined

### Variable Creation
Based on the provided data dictionary, we created two target variables, and three control Variables:
#### Target
 1. Y1 - Binary variable, set to 1 if the IPO offer price is less than the mid range of the IPO price
 2. Y2 - Binary variables, set to 1 if the IPO offer price is less than the first day trading price
 
#### Control
 1. C3' - Binary variable, set to 1 if Earnings per Share is positive
 2. C5' - Continuous/Float variable, representing the share overhang
 3. C6' - Continuous/Float variable, representing the up revision
 
 [New Variable Creation](https://github.com/ConorFeeney/IS540-Code/blob/master/Python%20for%20Comp%201/new_var.py) 

In [ ]:
calc(ipo_data)
del ipo_data['C3']
del ipo_data['C5']
del ipo_data['C6']
del ipo_data['T1']
del ipo_data['T2']
del ipo_data['T3']
del ipo_data['T4']
del ipo_data['T5']
del ipo_data['S1']
del ipo_data['S2']
del ipo_data['S3']
ipo_data.describe()

As we create several new variables, for example, we create T3x which is the ratio of Real Words to Total Words, we felt justified in removing the variable T3 from our data set as it would be covered in all future analysis by T3x. This was done to keep our data frame tidy.

As we read in our data, some variables were not read correctly in terms of types so we changed their type to be more appropriate and then printed the types of all our variables in order to ensure we were happy with them.

In [ ]:
ipo_data.C6x = ipo_data.C6x.astype(float)

ipo_data.C2 = ipo_data.C2.astype(int)
print(ipo_data.dtypes)

### Outliers
To deal with outliers, we plan on using the following methods for various data versions:
 1. Calculating the points that are greater than or less than 3 standard deviations
   away from the mean and setting any values outside this range to the upper / lower bound, respectively.
 2. Calculating the points that are greater than or less than 3 standard deviations
   away from the mean and setting any values outside this range to the mean
 3. Calculating the Interquartile Range and finding values outside the limits Q1-IQR*1.5 and Q3+IQR*1.5 
   and setting to be the mean
 4. Calculating the Interquartile Range and finding values outside the limits Q1-IQR*1.5 and Q3+IQR*1.5 
    and setting to be the the Q1 or Q3, respectively.

 [Outliers](https://github.com/ConorFeeney/IS540-Code/blob/master/Python%20for%20Comp%201/outliers.py) 

In [ ]:
# Put outlier code here

As we can see in the above script, we decided to use the fourth method for the purposes of this report. This was decided because it was felt that the standard deviation was too heavily influenced by extreme values, and additionally, while conducting Logistic Regression model tests, it was discovered that method four provided the highest F1-Score and AUC rating when coupled with other factors that will be discussed later.

### Recoding
During the data understanding step, we realized that I3, or the Standard Industry Code for each record, was not populated and/or incorrect for instances. After correcting these instances, we were able to match to an Industry Division based on the codes. This allowed us to recode the I3 column to a categorical variable of Industry divisions, that we will later be able to use as a way of clustering the records prior to modeling, to hopefully improve the model fit.

[Recoding SIC Column](https://github.com/ConorFeeney/IS540-Code/blob/master/Python%20for%20Comp%201/Recoding%20SIC%20Code%20to%20Industry%20Division.ipynb) 

### Correlation analysis
Next, we needed to select the predictor variables with low pair-wise correlation values. In order to do this, we used Spearman's correlation test to determine the statistical dependence between the rankings of pairs of variables.

In [3]:
ipo_data.corr(method='spearman').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('bwr'), axis=1)

### Standardization
To standardize, we developed code for three different methods:
 * Min/Max scaling
 * Zscore standardization
 * Decimal scaling
 
[Standardising Function](https://github.com/ConorFeeney/IS540-Code/blob/master/Python%20for%20Comp%201/Standardising.py) 

### Normalization

### Data Prep Summary
*After experimenting with each method in the suggested tasks, we decided the best approach for our model would be to utlize different combinations and orders of each in a series of models and evaluate performance along the way. In order to do this easily, we created functions for each step that will iterate through each method.*